In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data\data_draft\sample_bab4.csv', index_col = 0)

In [3]:
df.shape[0]

262

In [4]:
df_aman = df[df.label == 'aman'].copy()
df_berbahaya = df[df.label == 'berbahaya'].copy()

In [5]:
print(df_aman.shape[0])
print(df_berbahaya.shape[0])

131
131


In [6]:
split_df_aman = np.array_split(df_aman,10)
split_df_berbahaya = np.array_split(df_berbahaya,10)

In [7]:
split_df = []
for i in range(10):
    split_df.append(pd.concat([split_df_aman[i],split_df_berbahaya[i]]))
    # split_df[i].to_csv('data\data_draft\subset\subset{}.csv'.format(i+1))
        

In [8]:
import joblib
from sklearn.linear_model import SGDClassifier
from scipy.sparse import vstack
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [9]:
vect_desc = joblib.load(r'saved_model\vect_desc.pkl')
vect_title = joblib.load(r'saved_model\vect_title.pkl')
svm_desc = joblib.load(r'saved_model\SVM_desc.pkl')
svm_title = joblib.load(r'saved_model\SVM_title.pkl')

c:\Users\binta\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator TfidfTransformer from version 1.0.2 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\binta\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 1.0.2 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\binta\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator SGDClassifier from version 1.0.2 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own r

In [10]:
split_desc_tfidf = []
split_title_tfidf = []
for i in range(10):
    split_desc_tfidf.append(vect_desc.transform(split_df[i].desc_p.to_list()))
    split_title_tfidf.append(vect_title.transform(split_df[i].title_p.to_list()))

In [11]:
label = []
for i in range(10):
    label.append(split_df[i].label)
    

In [12]:
def decfunc_final(desc,title):
    desc_decfunc_value = 0.98/1.85
    title_decfunc_value = 0.87/1.85
    hasil = desc * desc_decfunc_value + title * title_decfunc_value 
    return "aman" if  hasil > 0 else "berbahaya"

In [13]:
acc = []
predicted_label_svm = []
for i in range(10):
    temp_label = label[:]
    test_label = temp_label.pop(i)
    train_label = pd.concat(temp_label[:])

    SVM_desc = SGDClassifier(loss='hinge')
    temp_split_desc_tfidf = split_desc_tfidf[:]
    test_desc = temp_split_desc_tfidf.pop(i)
    train_desc = vstack((temp_split_desc_tfidf[:]))
    SVM_desc.fit(train_desc,train_label)

    SVM_title = SGDClassifier(loss='hinge')
    temp_split_title_tfidf = split_title_tfidf[:]
    test_title = temp_split_title_tfidf.pop(i)
    train_title = vstack((temp_split_title_tfidf[:]))
    SVM_title.fit(train_title,train_label)

    dec_func_desc = SVM_desc.decision_function(test_desc)
    dec_func_title = SVM_title.decision_function(test_title)
    dec_func_desc = dec_func_desc*-1
    dec_func_title = dec_func_title*-1

    predic_final = list(map(decfunc_final,dec_func_desc,dec_func_title))
    predicted_label_svm.append(predic_final)
    acc.append(accuracy_score(predic_final,test_label))

In [34]:
acc[0]

0.9642857142857143

In [18]:
predicted_label_spm = []
acc_spm = []

from dodo_spm import spm

for i in range(10):
    predicted_label = []
    test_label = split_df[i]['label']
    for j in list(split_df[i]['url']):
        predicted_label.append(spm.profanity_check_url(j))
    predicted_label_spm.append(predicted_label)
    acc_spm.append(accuracy_score(predicted_label,test_label))

In [29]:
temp_subset1 = split_df[:]
test_subset1 = temp_subset1.pop(0)
train_subset1 = pd.concat(temp_subset1[:])
df_subset1_test = pd.DataFrame({'url':test_subset1.url, 'label' :test_subset1.label, 'SVM_label':predicted_label_svm[0], 'SPM_label':predicted_label_spm[0]})
df_subset1_train = pd.DataFrame({'url': train_subset1.url, 'label' : train_subset1.label})


In [31]:
df_subset1_train.to_csv('data\data_draft\subset1_train.csv')

In [32]:
df_subset1_test.to_csv('data\data_draft\subset1_test.csv')

In [33]:
acc_spm[0]

0.6785714285714286

In [36]:
akurasi_df = pd.DataFrame({'akurasi_svm':acc, 'akurasi_spm':acc_spm})

In [40]:
akurasi_df.to_csv(r'data\data_draft\akurasi_bab4.csv')

In [41]:
np.mean(acc)

0.9656593406593407

In [42]:
np.mean(acc_spm)

0.6601648351648352

In [ ]:
# train_label = pd.concat(label[0:9])
# test_label = label[9]

# SVM_desc = SGDClassifier(loss='hinge')
# train_desc = vstack((split_desc_tfidf[0:9]))
# test_desc = split_desc_tfidf[9]
# SVM_desc.fit(train_desc,train_label)

# SVM_title = SGDClassifier(loss='hinge')
# train_title = vstack((split_title_tfidf[0:9]))
# test_title = split_title_tfidf[9]
# SVM_title.fit(train_title,train_label)

# dec_func_desc = SVM_desc.decision_function(test_desc)
# dec_func_title = SVM_title.decision_function(test_title)
# dec_func_desc = dec_func_desc*-1
# dec_func_title = dec_func_title*-1

# predic_final = list(map(decfunc_final,dec_func_desc,dec_func_title))

# accuracy_score(predic_final,test_label)

# a = [1,2,3,4]

# temp = a[:]
# test = temp.pop(2)
# train = temp[:]